# 最終課題

In [ ]:
# This is the main file (240713)

In [4]:
!pip install wandb
!pip install omegaconf
!pip install termcolor
!pip install einops
!pip install git+https://github.com/openai/CLIP.git

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import numpy as np
import torch
from typing import Tuple
from termcolor import cprint
import sys
import torch.nn.functional as F
#from torchmetrics import Accuracy
#import hydra
from omegaconf import DictConfig
import wandb
from tqdm import tqdm

#filter
import scipy.signal
from sklearn.preprocessing import MinMaxScaler


import random

import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

import math

import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from einops.layers.torch import Rearrange
from einops import rearrange

import clip

from torchmetrics import Accuracy


#from src.datasets import ThingsMEGDataset
#from src.utils import set_seed

In [2]:

class ThingsMEGDataset(torch.utils.data.Dataset):
    def __init__(self, split: str, filtering: bool = True, data_dir: str = "data") -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.filtering = filtering
        self.num_classes = 1854

        self.X = torch.load(os.path.join(data_dir, f"{split}_X.pt"))[:,:,:-1]
        if filtering:
            for ii in tqdm(range(len(self.X))):
                self.X[ii] = self.filter(self.X[ii])

        self.subject_idxs = torch.load(os.path.join(data_dir, f"{split}_subject_idxs.pt"))

        if split in ["train", "val"]:
            self.y = torch.load(os.path.join(data_dir, f"{split}_y.pt"))
            assert len(torch.unique(self.y)) == self.num_classes, "Number of classes do not match."

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    def butter_bandpass(self, lowcut=0.5, highcut=50.0, fs=200, order=5):
        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = scipy.signal.butter(order, [low, high], btype='band')
        return b, a
    
    def bandpass_filter(self, data, lowcut=0.5, highcut=50.0, fs=200, order=5):
        b, a = self.butter_bandpass(lowcut, highcut, fs, order=order)
        y = scipy.signal.lfilter(b, a, data)
        return y

    def minmaxscl(self, data, feature_range=(-1, 1)):
        scaler = MinMaxScaler(feature_range)
        return scaler.fit_transform(data)

    def filter(self, x):
        x = self.bandpass_filter(x)
        x = self.minmaxscl(x)
        return torch.Tensor(x)
    
    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]




class ThingsMEGDataset_selective(torch.utils.data.Dataset):
    def __init__(self, split: str, sj_idx: None, filtering: bool = True, data_dir: str = "data") -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.filtering = filtering
        self.num_classes = 1854

        self.X = torch.load(os.path.join(data_dir, f"{split}_X.pt"))[:,:,:-1]
        self.subject_idxs = torch.load(os.path.join(data_dir, f"{split}_subject_idxs.pt"))

        
        if split in ["train", "val"]:
            self.y = torch.load(os.path.join(data_dir, f"{split}_y.pt"))
            assert len(torch.unique(self.y)) == self.num_classes, "Number of classes do not match."
            if sj_idx is not None:
                self.sjselected = torch.nonzero(self.subject_idxs == sj_idx).squeeze()
                self.X = self.X[self.sjselected]
                self.subject_idxs = self.subject_idxs[self.sjselected]
                self.y = self.y[self.sjselected]
                
        if filtering:
            for ii in tqdm(range(len(self.X))):
                self.X[ii] = self.filter(self.X[ii])

    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i]
        else:
            return self.X[i], self.subject_idxs[i]

    def butter_bandpass(self, lowcut=0.5, highcut=50.0, fs=200, order=5):
        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = scipy.signal.butter(order, [low, high], btype='band')
        return b, a
    
    def bandpass_filter(self, data, lowcut=0.5, highcut=50.0, fs=200, order=5):
        b, a = self.butter_bandpass(lowcut, highcut, fs, order=order)
        y = scipy.signal.lfilter(b, a, data)
        return y

    def minmaxscl(self, data, feature_range=(-1, 1)):
        scaler = MinMaxScaler(feature_range)
        return scaler.fit_transform(data)

    def filter(self, x):
        x = self.bandpass_filter(x)
        x = self.minmaxscl(x)
        return torch.Tensor(x)
    
    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]



class ThingsMEGDataset5_PreloadSplit(torch.utils.data.Dataset):
    def __init__(self, preprocess, split: str, sj_idx: int=None, filtering: bool = True, data_dir: str = "data", img_dir: str="images-002/images/", clip_model: str="RN50", save_dir: str="save_imgs/", device: str="cuda" if torch.cuda.is_available() else "cpu", just_load: bool=True) -> None:
        super().__init__()

        assert split in ["train", "val", "test"], f"Invalid split: {split}"
        self.split = split
        self.filtering = filtering
        self.img_dir = img_dir
        self.num_classes = 1854

        self.X = torch.load(os.path.join(data_dir, f"{split}_X.pt"))[:,:,:-1]
        self.subject_idxs = torch.load(os.path.join(data_dir, f"{split}_subject_idxs.pt"))

        self.device = device
        #self.preprocess = preprocess
        self.CLIP, self.preprocess = clip.load(clip_model, device=self.device)
        
        if split in ["train", "val"]:
            if sj_idx is not None:
                self.sjselected = torch.nonzero(self.subject_idxs == sj_idx).squeeze()
                self.X = self.X[self.sjselected]
                self.subject_idxs = self.subject_idxs[self.sjselected]
            self.y = torch.load(os.path.join(data_dir, f"{split}_y.pt"))
            assert len(torch.unique(self.y)) == self.num_classes, "Number of classes do not match."
            
        if filtering:
            for ii in tqdm(range(len(self.X)), "Filtering for "+split):
                self.X[ii] = self.filter(self.X[ii])
        
        if hasattr(self, "y"):
            f = open(os.path.join(data_dir, f"{split}_image_paths.txt"))
            self.paths = []
            while True:
              line = f.readline()
              if line == '':
                break
              self.paths.append(line.strip())
            f.close()

            self.paths = [self.paths[idx] for idx in self.sjselected]
            separator = 500
            if not just_load:
                flg = True
                for ii, path in tqdm(enumerate(self.paths), "Compressing images for "+split):
                    if flg:
                        self.images = self.preprocess(Image.open(os.path.join(self.img_dir, path))).unsqueeze(0)
                        flg = False
                    else:
                        self.images = torch.cat((self.images, self.preprocess(Image.open(os.path.join(self.img_dir, path))).unsqueeze(0)),dim=0)
                    if (ii+1)%separator==0:
                        with torch.no_grad():
                            self.img_features = self.CLIP.encode_image(self.images.to(device))
                        torch.save(self.img_features, os.path.join(save_dir,self.split+str((ii+1)//separator)+".pt"))
                        flg = True
                
                if (ii+1)%separator!=0:
                    with torch.no_grad():
                        self.img_features = self.CLIP.encode_image(self.images.to(device))
                    torch.save(self.img_features, os.path.join(save_dir,self.split+str((ii+1)//separator+1)+".pt"))
                num_loaded = ii
            else:
                num_loaded = len(self.paths)
                
            
            if (num_loaded+1)%separator!=0:
                flg = True
                for ii in range((num_loaded+1)//separator+1):
                    if flg:
                        self.images_clip = torch.load(os.path.join(save_dir,self.split+str(ii+1)+".pt"))
                        flg = False
                    else:
                        self.images_clip = torch.cat(( self.images_clip, torch.load(os.path.join(save_dir,self.split+str(ii+1)+".pt")) ), dim=0)
            else:
                flg = True
                for ii in range((num_loaded+1)//separator):
                    if flg:
                        self.images_clip = torch.load(os.path.join(save_dir,self.split+str(ii+1)+".pt"))
                        flg = False
                    else:
                        self.images_clip = torch.cat(( self.images_clip, torch.load(os.path.join(save_dir,self.split+str(ii+1)+".pt")) ), dim=0)
        
    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, i):
        if hasattr(self, "y"):
            return self.X[i], self.y[i], self.subject_idxs[i], self.images_clip[i]
        else:
            return self.X[i], self.subject_idxs[i]

    def butter_bandpass(self, lowcut=0.5, highcut=50.0, fs=200, order=5):
        nyquist = 0.5 * fs
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = scipy.signal.butter(order, [low, high], btype='band')
        return b, a
    
    def bandpass_filter(self, data, lowcut=0.5, highcut=50.0, fs=200, order=5):
        b, a = self.butter_bandpass(lowcut, highcut, fs, order=order)
        y = scipy.signal.lfilter(b, a, data)
        return y

    def minmaxscl(self, data, feature_range=(-1, 1)):
        scaler = MinMaxScaler(feature_range)
        return scaler.fit_transform(data)

    def filter(self, x):
        x = self.bandpass_filter(x)
        x = self.minmaxscl(x)
        return torch.Tensor(x)
    
    @property
    def num_channels(self) -> int:
        return self.X.shape[1]

    @property
    def seq_len(self) -> int:
        return self.X.shape[2]

    

# MAE

In [10]:
args = {"batch_size": 256, "epochs": 80, "lr": 0.001, "device": "cuda", "num_workers": 0, "seed": 1234, "use_wandb": False, "data_dir": "data/", "model_path": "A"}


#set_seed(args.seed)
#logdir = hydra.core.hydra_config.HydraConfig.get().runtime.output_dir

#if args.use_wandb:
#    wandb.init(mode="online", dir=logdir, project="MEG-classification")

# ------------------
#    Dataloader
# ------------------
loader_args = {"batch_size": args["batch_size"], "num_workers": args["num_workers"]}

train_set = ThingsMEGDataset("train", args["data_dir"])
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
val_set = ThingsMEGDataset("val", args["data_dir"])
val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)
test_set = ThingsMEGDataset("test", args["data_dir"])
test_loader = torch.utils.data.DataLoader(
    test_set, shuffle=False, batch_size=args["batch_size"], num_workers=args["num_workers"]
)


100%|███████████████████████████████████████████████████████████████████████████| 16432/16432 [00:17<00:00, 940.75it/s]


In [3]:
#240707 Change this block to fit the dataset to use them as MAE input

def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


fix_seed(seed=42)


def random_indexes(size):
    """
    パッチをランダムに並べ替えるためのindexを生成する関数．

    Argument
    --------
    size : int
        入力されるパッチの数（系列長Nと同じ値）．
    """
    forward_indexes = np.arange(size)  # 0からsizeまでを並べた配列を作成
    np.random.shuffle(forward_indexes)  # 生成した配列をシャッフルすることで，パッチの順番をランダムに決定
    backward_indexes = np.argsort(forward_indexes)  # 並べ替えたパッチをもとの順番に戻すためのidx

    return forward_indexes, backward_indexes


def take_indexes(sequences, indexes):
    """
    パッチを並べ替えるための関数．

    Argument
    --------
    sequences : torch.Tensor
        入力画像をパッチ分割したデータ．(B, N, dim)の形状をしている．
    indexes : np.ndarray
        並べ替えるために利用するindex．
        random_indexesで生成したforward_indexesかbackward_indexesが入ることが想定されている．
    """
    return torch.gather(sequences, dim=1, index=indexes.unsqueeze(2).repeat(1, 1, sequences.shape[-1]))


############################################################################################

class Attention(nn.Module):
    def __init__(self, dim, heads, dim_head, dropout=0.):
        """
        Arguments
        ---------
        dim : int
            入力データの次元数．埋め込み次元数と一致する．
        heads : int
            ヘッドの数．
        dim_head : int
            各ヘッドのデータの次元数．
        dropout : float
            Dropoutの確率(default=0.)．
        """
        super().__init__()

        self.dim = dim
        self.dim_head = dim_head
        inner_dim = dim_head * heads  # ヘッドに分割する前のQ, K, Vの次元数．self.dimと異なっても良い．
        project_out = not (heads == 1 and dim_head == dim)  # headsが1，dim_headがdimと等しければ通常のSelf-Attention

        self.heads = heads
        self.scale = math.sqrt(dim_head)  # ソフトマックス関数を適用する前のスケーリング係数(dim_k)

        self.attend = nn.Softmax(dim=-1)  # アテンションスコアの算出に利用するソフトマックス関数
        self.dropout = nn.Dropout(dropout)

        # Q, K, Vに変換するための全結合層
        self.to_q = nn.Linear(in_features=dim, out_features=inner_dim)
        self.to_k = nn.Linear(in_features=dim, out_features=inner_dim)
        self.to_v = nn.Linear(in_features=dim, out_features=inner_dim)

        # dim != inner_dimなら線形層を入れる，そうでなければそのまま出力
        self.to_out = nn.Sequential(
            nn.Linear(in_features=inner_dim, out_features=dim),
            nn.Dropout(dropout),
        ) if project_out else nn.Identity()

    def forward(self, x):
        """
        B: バッチサイズ
        N: 系列長
        D: データの次元数(dim)
        """
        B, N, D = x.size()

        # 入力データをQ, K, Vに変換する
        # (B, N, dim) -> (B, N, inner_dim)
        q = self.to_q(x)
        k = self.to_k(x)
        v = self.to_v(x)

        # Q, K, Vをヘッドに分割する
        # (B, N, inner_dim) -> (B, heads, N, dim_head)
        q = rearrange(q, "b n (h d) -> b h n d", h=self.heads, d=self.dim_head)
        k = rearrange(k, "b n (h d) -> b h n d", h=self.heads, d=self.dim_head)
        v = rearrange(v, "b n (h d) -> b h n d", h=self.heads, d=self.dim_head)

        # QK^T / sqrt(d_k)を計算する
        # (B, heads, N, dim_head) x (B, heads, dim_head, N) -> (B, heads, N, N)
        dots = torch.matmul(q, k.transpose(-2, -1)) / self.scale

        # ソフトマックス関数でスコアを算出し，Dropoutをする
        attn = self.attend(dots)
        attn = self.dropout(attn)

        # softmax(QK^T / sqrt(d_k))Vを計算する
        # (B, heads, N, N) x (B, heads, N, dim_head) -> (B, heads, N, dim_head)
        out = torch.matmul(attn ,v)

        # もとの形に戻す
        # (B, heads, N, dim_head) -> (B, N, dim)
        out = rearrange(out, "b h n d -> b n (h d)", h=self.heads, d=self.dim_head)

        # 次元が違っていればもとに戻して出力
        # 表現の可視化のためにattention mapも返すようにしておく
        return self.to_out(out), attn

############################################################################################

class FFN(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        """
        Arguments
        ---------
        dim : int
            入力データの次元数．
        hidden_dim : int
            隠れ層の次元．
        dropout : float
            各全結合層の後のDropoutの確率(default=0.)．
        """
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(in_features=dim, out_features=hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(in_features=hidden_dim, out_features=dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        """
        (B, D) -> (B, D)
        B: バッチサイズ
        D: 次元数
        """
        return self.net(x)


############################################################################################

class Block(nn.Module):
    def __init__(self, dim, heads, dim_head, mlp_dim, dropout):
        """
        TransformerのEncoder Blockの実装．

        Arguments
        ---------
        dim : int
            埋め込みされた次元数．PatchEmbedのembed_dimと同じ値．
        heads : int
            Multi-Head Attentionのヘッドの数．
        dim_head : int
            Multi-Head Attentionの各ヘッドの次元数．
        mlp_dim : int
            Feed-Forward Networkの隠れ層の次元数．
        dropout : float
            Droptou層の確率p．
        """
        super().__init__()

        self.attn_ln = nn.LayerNorm(dim)  # Attention前のLayerNorm
        self.attn = Attention(dim, heads, dim_head, dropout)
        self.ffn_ln = nn.LayerNorm(dim)  # FFN前のLayerNorm
        self.ffn = FFN(dim, mlp_dim, dropout)

    def forward(self, x, return_attn=False):
        """
        x: (B, N, dim)
        B: バッチサイズ
        N: 系列長
        dim: 埋め込み次元
        """
        y, attn = self.attn(self.attn_ln(x))
        if return_attn:  # attention mapを返す（attention mapの可視化に利用）
            return attn
        x = y + x
        out = self.ffn(self.ffn_ln(x)) + x

        return out

############################################################################################

class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim):
        """
        入力画像をパッチごとに埋め込むための層．

        Arguments
        ---------
        image_size : Tuple[int]
            入力画像のサイズ．
        patch_size : Tuple[int]
            各パッチのサイズ．
        in_channels : int
            入力画像のチャネル数．
        embed_dim : int
            埋め込み後の次元数．
        """
        super().__init__()

        image_height, image_width = image_size
        patch_height, patch_width = patch_size

        #print(image_height, patch_height, image_width, patch_width)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, "パッチサイズは，入力画像のサイズを割り切れる必要があります．"

        num_patches = (image_height // patch_height) * (image_width // patch_width)  # パッチの数
        patch_dim = in_channels * patch_height * patch_width  # 各パッチを平坦化したときの次元数

        self.to_patch_embedding = nn.Sequential(
            Rearrange("b c (h p1) (w p2) -> b (h w) (p1 p2 c)", p1=patch_height, p2=patch_width),  # 画像をパッチに分割して平坦化
            nn.Linear(in_features=patch_dim, out_features=embed_dim),  # 埋め込みを行う
        )

    def forward(self, x):
        """
        B: バッチサイズ
        C: 入力画像のチャネル数
        H: 入力画像の高さ
        W: 入力画像の幅
        """
        return self.to_patch_embedding(x)  # (B, C, H, W) -> (B, num_patches, embed_dim)

############################################################################################

class PatchShuffle(nn.Module):
    def __init__(self, ratio):
        # ratio: Encoderに入力しないパッチの割合
        super().__init__()
        self.ratio = ratio

    def forward(self, patches):
        """
        B: バッチサイズ
        N: 系列長（＝パッチの数）
        dim: 次元数（＝埋め込みの次元数）
        """
        B, N, dim = patches.shape
        remain_N = int(N * (1 - self.ratio))  # Encoderに入力するパッチの数

        indexes = [random_indexes(N) for _ in range(B)]  # バッチごとに異なる順番のindexを作る
        forward_indexes = torch.as_tensor(np.stack([i[0] for i in indexes], axis=-1), dtype=torch.long).T.to(patches.device)  # バッチを並べ替えるときのidx (B, N)
        backward_indexes = torch.as_tensor(np.stack([i[1] for i in indexes], axis=-1), dtype=torch.long).T.to(patches.device)  # 並べ替えたパッチをもとの順番に戻すためのidx  (B, N)

        patches = take_indexes(patches, forward_indexes)  # パッチを並べ替える
        patches = patches[:, :remain_N, :]  # Encoderに入力するパッチを抽出

        return patches, forward_indexes, backward_indexes


    def random_indexes(size):
        """
        パッチをランダムに並べ替えるためのindexを生成する関数．

        Argument
        --------
        size : int
            入力されるパッチの数（系列長Nと同じ値）．
        """
        forward_indexes = np.arange(size)  # 0からsizeまでを並べた配列を作成
        np.random.shuffle(forward_indexes)  # 生成した配列をシャッフルすることで，パッチの順番をランダムに決定
        backward_indexes = np.argsort(forward_indexes)  # 並べ替えたパッチをもとの順番に戻すためのidx

        return forward_indexes, backward_indexes



    def take_indexes(sequences, indexes):
        """
        パッチを並べ替えるための関数．

        Argument
        --------
        sequences : torch.Tensor
            入力画像をパッチ分割したデータ．(B, N, dim)の形状をしている．
        indexes : np.ndarray
            並べ替えるために利用するindex．
            random_indexesで生成したforward_indexesかbackward_indexesが入ることが想定されている．
        """
        return torch.gather(sequences, dim=1, index=indexes.unsqueeze(2).repeat(1, 1, sequences.shape[-1]))

############################################################################################

class MAE_Encoder(nn.Module):
    def __init__(self, image_size=[272, 286], patch_size=[34, 13], emb_dim=192, num_layer=12,
                 heads=3, dim_head=64, mlp_dim=192, mask_ratio=0.75, dropout=0.):
        """
        Arguments
        ---------

        image_size : List[int]
            入力画像の大きさ．
        patch_size : List[int]
            各パッチの大きさ．
        emb_dim : int
            データを埋め込む次元の数．
        num_layer : int
            Encoderに含まれるBlockの数．
        heads : int
            Multi-Head Attentionのヘッドの数．
        dim_head : int
            Multi-Head Attentionの各ヘッドの次元数．
        mlp_dim : int
            Feed-Forward Networkの隠れ層の次元数．
        mask_ratio : float
            入力パッチのマスクする割合．
        dropout : float
            ドロップアウトの確率．
        """
        super().__init__()
        img_height, img_width = image_size
        patch_height, patch_width = patch_size
        num_patches = (img_height // patch_height) * (img_width // patch_width)

        self.cls_token = torch.nn.Parameter(torch.randn(1, 1, emb_dim))  # class tokenの初期化
        self.pos_embedding = torch.nn.Parameter(torch.randn(1, num_patches, emb_dim))  # positional embedding（学習可能にしている）
        self.shuffle = PatchShuffle(mask_ratio)

        # 入力画像をパッチに分割する
        self.patchify = PatchEmbedding(image_size, patch_size, 1, emb_dim)

        # Encoder（Blockを重ねる）
        self.transformer = torch.nn.Sequential(*[Block(emb_dim, heads, dim_head, mlp_dim, dropout) for _ in range(num_layer)])

        self.layer_norm = nn.LayerNorm(emb_dim)

        self.init_weight()

    def init_weight(self):
        torch.nn.init.normal_(self.cls_token, std=0.02)
        torch.nn.init.normal_(self.pos_embedding, std=0.02)

    def forward(self, img):
        # 1. 入力画像をパッチに分割して，positional embeddingする
        patches = self.patchify(img)
        patches = patches + self.pos_embedding

        # 2. 分割したパッチをランダムに並べ替えて，必要なパッチのみ得る
        patches, forward_indexes, backward_indexes = self.shuffle(patches)

        # class tokenを結合
        patches = torch.cat([self.cls_token.repeat(patches.shape[0], 1, 1), patches], dim=1)

        # 3. Encoderで入力データを処理する
        features = self.layer_norm(self.transformer(patches))

        return features, backward_indexes


############################################################################################

class MAE_Decoder(nn.Module):
    def __init__(self, image_size=[272, 286], patch_size=[34, 13], emb_dim=192, num_layer=4,
                 heads=3, dim_head=64, mlp_dim=192, dropout=0.):
        """
        Arguments
        ---------

        image_size : List[int]
            入力画像の大きさ．
        patch_size : List[int]
            各パッチの大きさ．
        emb_dim : int
            データを埋め込む次元の数．
        num_layer : int
            Decoderに含まれるBlockの数．
        heads : int
            Multi-Head Attentionのヘッドの数．
        dim_head : int
            Multi-Head Attentionの各ヘッドの次元数．
        mlp_dim : int
            Feed-Forward Networkの隠れ層の次元数．
        dropout : float
            ドロップアウトの確率．
        """
        super().__init__()
        img_height, img_width = image_size
        patch_height, patch_width = patch_size
        num_patches = (img_height // patch_height) * (img_width // patch_width)

        self.mask_token = torch.nn.Parameter(torch.rand(1, 1, emb_dim))
        self.pos_embedding = torch.nn.Parameter(torch.rand(1, num_patches+1, emb_dim))

        # Decoder(Blockを重ねる）
        self.transformer = torch.nn.Sequential(*[Block(emb_dim, heads, dim_head, mlp_dim, dropout) for _ in range(num_layer)])

        # 埋め込みされた表現から画像を復元するためのhead
        self.head = torch.nn.Linear(emb_dim, 1 * patch_height * patch_width)
        # (B, N, dim)から(B, C, H, W)にreshapeするためのインスタンス
        self.patch2img = Rearrange("b (h w) (c p1 p2) -> b c (h p1) (w p2)", p1=patch_height, p2=patch_width, h=img_height // patch_height)

        self.init_weight()

    def init_weight(self):
        torch.nn.init.normal_(self.mask_token, std=0.02)
        torch.nn.init.normal_(self.pos_embedding, std=0.02)

    def forward(self, features, backward_indexes):
        # 系列長
        T = features.shape[1]

        # class tokenがある分backward_indexesの最初に0を追加する
        # .toはデバイスの変更でよく利用するが，tensorを渡すことでdtypeを変えることができる
        backward_indexes = torch.cat([torch.zeros(backward_indexes.shape[0], 1).to(backward_indexes), backward_indexes+1], dim=1)

        # 1. mask_tokenを結合して並べ替える．
        # (B, N*(1-mask_ratio)+1, dim) -> (B, N+1, dim)
        features = torch.cat([features, self.mask_token.repeat(features.shape[0], backward_indexes.shape[1] - features.shape[1], 1)], dim=1)
        features = take_indexes(features, backward_indexes)
        features = features + self.pos_embedding

        features = self.transformer(features)

        # class tokenを除去する
        # (B, N+1, dim) -> (B, N, dim)
        features = features[:, 1:, :]

        # 2. 画像を再構成する．
        # (B, N, dim) -> (B, N, 3 * patch_height * patch_width)
        patches = self.head(features)

        # MAEではマスクした部分でのみ損失関数を計算するため，maskも一緒に返す
        mask = torch.zeros_like(patches)
        mask[:, T-1:] = 1  # cls tokenを含めていた分ずらしている
        mask = take_indexes(mask, backward_indexes[:, 1:] - 1)

        img = self.patch2img(patches)
        mask = self.patch2img(mask)

        return img, mask


############################################################################################

class MAE_ViT(nn.Module):
    def __init__(self, image_size=[272, 286], patch_size=[34, 13], emb_dim=192,
                 enc_layers=12, enc_heads=3, enc_dim_head=64, enc_mlp_dim=768,
                 dec_layers=4, dec_heads=3, dec_dim_head=64, dec_mlp_dim=768,
                 mask_ratio=0.75, dropout=0.):
        """
        Arguments
        ---------
        image_size : List[int]
            入力画像の大きさ．
        patch_size : List[int]
            各パッチの大きさ．
        emb_dim : int
            データを埋め込む次元の数．
        {enc/dec}_layers : int
            Encoder / Decoderに含まれるBlockの数．
        {enc/dec}_heads : int
            Encoder / DecoderのMulti-Head Attentionのヘッドの数．
        {enc/dec}_dim_head : int
            Encoder / DecoderのMulti-Head Attentionの各ヘッドの次元数．
        {enc/dec}_mlp_dim : int
            Encoder / DecoderのFeed-Forward Networkの隠れ層の次元数．
        mask_ratio : float
            入力パッチのマスクする割合．
        dropout : float
            ドロップアウトの確率．
        """
        super().__init__()

        self.encoder = MAE_Encoder(image_size, patch_size, emb_dim, enc_layers,
                                   enc_heads, enc_dim_head, enc_mlp_dim, mask_ratio, dropout)
        self.decoder = MAE_Decoder(image_size, patch_size, emb_dim, dec_layers,
                                   dec_heads, dec_dim_head, dec_mlp_dim, dropout)

    def forward(self, img):
        features, backward_indexes = self.encoder(img)
        rec_img, mask = self.decoder(features, backward_indexes)
        return rec_img, mask

    def get_last_selfattention(self, x):
        patches = self.encoder.patchify(x)
        patches = patches + self.encoder.pos_embedding

        patches = torch.cat([self.encoder.cls_token.repeat(patches.shape[0], 1, 1), patches], dim=1)  # class tokenを結合
        for i, block in enumerate(self.encoder.transformer):
            if i < len(self.encoder.transformer) - 1:
                patches = block(patches)
            else:
                return block(patches, return_attn=True)



class Projection_Layer(nn.Module):
    def __init__(self, in_channels, out_channels, data_width):
        """
        Arguments
        ---------
        in_channels : int
            number of input channels．
        out_channels : int
            number of output channels
        data_width : int
            width of data
        """
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv1d(in_channels =in_channels,out_channels=out_channels,kernel_size=1,stride=1),
            nn.GELU(),
            nn.Flatten(),
            nn.Linear(out_channels*data_width, 1024),
            nn.ReLU(),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        """
        (B, D) -> (B, D)
        B: バッチサイズ
        D: 次元数
        """
        return self.net(x)


class Projection_LayerL(nn.Module):
    def __init__(self, in_channels, out_channels, data_width):
        """
        Arguments
        ---------
        in_channels : int
            number of input channels．
        out_channels : int
            number of output channels
        data_width : int
            width of data
        """
        super().__init__()

        """
        self.net = nn.Sequential(
            nn.Conv1d(in_channels =in_channels,out_channels=out_channels,kernel_size=1,stride=1),
            nn.GELU(),
            nn.Flatten(),
            nn.Linear(out_channels*data_width, 1024),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        """

        self.net = nn.Sequential(
            #input dim, output dim (num filters), size of filter (square)
            nn.Conv2d(in_channels, 64, 3,padding='same'),              # 32x32x1 -> 32x32x64
            nn.ReLU(),
            nn.Conv2d(64, 64, 3,padding='same'),              # 32x32x64 -> 32x32x64
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3,padding='same'),             # 32x32x64 -> 32x32x64
            nn.ReLU(),
            nn.MaxPool2d(2),                                 # 32x32x64 -> 16x16x64
            nn.Dropout(0.25),
        
            nn.Conv2d(64, 128, 3,padding='same'),              # 16x16x64 -> 16x16x128
            nn.ReLU(),
            nn.Conv2d(128, 128, 3,padding='same'),             # 16x16x128 -> 16x16x128
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3,padding='same'),             # 16x16x128 -> 16x16x128
            nn.ReLU(),
            nn.MaxPool2d(2),                                   # 16x16x128 -> 8x8x128
            nn.Dropout(0.25),
        
            nn.Conv2d(128, 256, 3,padding='same'),              # 8x8x128 -> 8x8x256
            nn.ReLU(),
            nn.Conv2d(256, 256, 3,padding='same'),              # 8x8x256 -> 8x8x256
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3,padding='same'),             # 8x8x256 -> 8x8x256
            nn.ReLU(),
            nn.Conv2d(256, 256, 3,padding='same'),             # 8x8x256 -> 8x8x256
            nn.ReLU(),
            nn.Conv2d(256, 256, 3,padding='same'),             # 8x8x256 -> 8x8x256
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 3,padding='same'),             # 8x8x512 -> 8x8x512
            nn.ReLU(),
            nn.Conv2d(512, 512, 3,padding='same'),             # 8x8x512 -> 8x8x512
            nn.ReLU(),
            nn.AvgPool2d(8),                  # 8x8x512 -> 1x1x512
            
            nn.Flatten(),
            nn.Linear(15360, 1024),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(1024, 1024)
        )

    def forward(self, x):
        """
        (B, D) -> (B, D)
        B: バッチサイズ
        D: 次元数
        """
        return self.net(x)



class Projection_LayerS(nn.Module):
    def __init__(self, in_channels, out_channels, data_width):
        """
        Arguments
        ---------
        in_channels : int
            number of input channels．
        out_channels : int
            number of output channels
        data_width : int
            width of data
        """
        super().__init__()

        self.net = nn.Sequential(
            #input dim, output dim (num filters), size of filter (square)
            nn.Conv2d(in_channels, 8, 3,padding='same'),              # 32x32x1 -> 32x32x64
            nn.ReLU(),
            nn.Conv2d(8, 8, 3,padding='same'),              # 32x32x64 -> 32x32x64
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(8, 8, 3,padding='same'),             # 32x32x64 -> 32x32x64
            nn.ReLU(),
            nn.AvgPool2d(10),                  # 8x8x512 -> 1x1x512
                 
            nn.Flatten(),
            nn.Linear(20672, 20672//10),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(20672//10, 1024)
        )

    def forward(self, x):
        """
        (B, D) -> (B, D)
        B: バッチサイズ
        D: 次元数
        """
        return self.net(x)



class BasicConvClassifier(nn.Module):
    def __init__(
        self,
        num_classes: int,
        seq_len: int,
        in_channels: int,
        hid_dim: int = 1024
    ) -> None:
        super().__init__()

        self.blocks = nn.Sequential(
            ConvBlock(in_channels, hid_dim),
            ConvBlock(hid_dim, hid_dim),
            ConvBlock(hid_dim, hid_dim)
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            Rearrange("b d 1 -> b d"),
            nn.Linear(hid_dim, hid_dim*2),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hid_dim*2, hid_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hid_dim, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """_summary_
        Args:
            X ( b, c, t ): _description_
        Returns:
            X ( b, num_classes ): _description_
        """
        X = self.blocks(X)

        return self.head(X)





class BasicConvProjector(nn.Module):
    def __init__(
        self,
        num_classes: int,
        seq_len: int,
        in_channels: int,
        hid_dim: int = 1024
    ) -> None:
        super().__init__()

        self.blocks = nn.Sequential(
            ConvBlock(in_channels, hid_dim),
            ConvBlock(hid_dim, hid_dim),
            ConvBlock(hid_dim, hid_dim)
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            Rearrange("b d 1 -> b d"),
            nn.Linear(hid_dim, hid_dim*2),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hid_dim*2, hid_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hid_dim, num_classes),
            nn.ReLU(),
            nn.Dropout(0.5),
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """_summary_
        Args:
            X ( b, c, t ): _description_
        Returns:
            X ( b, num_classes ): _description_
        """
        X = self.blocks(X)

        return self.head(X)




class ConvBlock(nn.Module):
    def __init__(
        self,
        in_dim,
        out_dim,
        kernel_size: int = 3,
        p_drop: float = 0.50,
    ) -> None:
        super().__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim

        self.conv0 = nn.Conv1d(in_dim, out_dim, kernel_size, padding="same")
        self.conv1 = nn.Conv1d(out_dim, out_dim, kernel_size, padding="same")
        # self.conv2 = nn.Conv1d(out_dim, out_dim, kernel_size) # , padding="same")
        
        self.batchnorm0 = nn.BatchNorm1d(num_features=out_dim)
        self.batchnorm1 = nn.BatchNorm1d(num_features=out_dim)

        self.dropout = nn.Dropout(p_drop)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        if self.in_dim == self.out_dim:
            X = self.conv0(X) + X  # skip connection
        else:
            X = self.conv0(X)

        X = F.gelu(self.batchnorm0(X))

        X = self.conv1(X) + X  # skip connection
        X = F.gelu(self.batchnorm1(X))

        # X = self.conv2(X)
        # X = F.glu(X, dim=-2)

        return self.dropout(X)




class BasicConvClassifier_original(nn.Module):
    def __init__(
        self,
        num_classes: int,
        seq_len: int,
        in_channels: int,
        hid_dim: int = 128
    ) -> None:
        super().__init__()

        self.blocks = nn.Sequential(
            ConvBlock_original(in_channels, hid_dim),
            ConvBlock_original(hid_dim, hid_dim),
            ConvBlock_original(hid_dim, hid_dim),
        )

        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            Rearrange("b d 1 -> b d"),
            nn.Linear(hid_dim, hid_dim),
            nn.Linear(hid_dim, num_classes),
        )

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """_summary_
        Args:
            X ( b, c, t ): _description_
        Returns:
            X ( b, num_classes ): _description_
        """
        X = self.blocks(X)

        return self.head(X)


class ConvBlock_original(nn.Module):
    def __init__(
        self,
        in_dim,
        out_dim,
        kernel_size: int = 3,
        p_drop: float = 0.1,
    ) -> None:
        super().__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim

        self.conv0 = nn.Conv1d(in_dim, out_dim, kernel_size, padding="same")
        self.conv1 = nn.Conv1d(out_dim, out_dim, kernel_size, padding="same")
        # self.conv2 = nn.Conv1d(out_dim, out_dim, kernel_size) # , padding="same")
        
        self.batchnorm0 = nn.BatchNorm1d(num_features=out_dim)
        self.batchnorm1 = nn.BatchNorm1d(num_features=out_dim)

        self.dropout = nn.Dropout(p_drop)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        if self.in_dim == self.out_dim:
            X = self.conv0(X) + X  # skip connection
        else:
            X = self.conv0(X)

        X = F.gelu(self.batchnorm0(X))

        X = self.conv1(X) + X  # skip connection
        X = F.gelu(self.batchnorm1(X))

        # X = self.conv2(X)
        # X = F.glu(X, dim=-2)

        return self.dropout(X)

In [4]:
class CosineScheduler:
    def __init__(self, epochs, lr, warmup_length=5):
        """
        Arguments
        ---------
        epochs : int
            学習のエポック数．
        lr : float
            学習率．
        warmup_length : int
            warmupを適用するエポック数．
        """
        self.epochs = epochs
        self.lr = lr
        self.warmup = warmup_length

    def __call__(self, epoch):
        """
        Arguments
        ---------
        epoch : int
            現在のエポック数．
        """
        progress = (epoch - self.warmup) / (self.epochs - self.warmup)
        progress = np.clip(progress, 0.0, 1.0)
        lr = self.lr * 0.5 * (1. + np.cos(np.pi * progress))

        if self.warmup:
            lr = lr * min(1., (epoch+1) / self.warmup)

        return lr

In [5]:
def set_lr(lr, optimizer):
  for param_group in optimizer.param_groups:
      param_group["lr"] = lr

In [6]:
def adjustment(x):
    sz = x.size()
    x_list = []
    for spl in x:
        x_ = torch.cat((spl, torch.zeros(spl.size(0), 5)), dim=1)
        avg_cnl = torch.mean(spl, dim=0, keepdim=True)
        avg_cnl = torch.cat((avg_cnl, torch.zeros(1, 5)), dim=1)
        x_ = torch.cat((x_,avg_cnl), dim=0)
        x_list.append(x_)
    
    x_list = torch.stack(x_list)
    x_list = torch.unsqueeze(x_list, dim=1)
    
    return x_list

In [25]:
# ハイパーパラメータの設定
config = {
    "image_size": [271, 280],
    "patch_size": [271, 20],
    "emb_dim": 1854,
    "enc_layers": 24,
    "enc_heads": 8,
    "enc_dim_head": 16,
    "enc_mlp_dim": 1854,
    "dec_layers": 4,
    "dec_heads": 5,
    "dec_dim_head": 16,
    "dec_mlp_dim": 1854,
    "mask_ratio": 0.4,
    "dropout": 0.25
}

device = "cuda" if torch.cuda.is_available() else "cpu"
model = MAE_ViT(**config).to(device)
epochs = 100
lr = 0.0004
step_count = 0
warmup_length = 25
batch_size = 128
optimizer = optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.95), weight_decay=0.05)
scheduler = CosineScheduler(epochs, lr, warmup_length)

In [26]:
for epoch in range(epochs):
    # スケジューラで学習率を更新する
    new_lr = scheduler(epoch)
    set_lr(new_lr, optimizer)

    total_train_loss = 0.
    total_valid_loss = 0.

    # モデルの訓練
    for x, _, _ in tqdm(train_loader):
        x = x.unsqueeze(1)
        step_count += 1
        model.train()
        x = x.to(device)

        rec_img, mask = model(x)
        train_loss = torch.mean((rec_img - x) ** 2 * mask) / config["mask_ratio"]
        train_loss.backward()

        if step_count % 8 == 0:  # 8イテレーションごとに更新することで，擬似的にバッチサイズを大きくしている
            optimizer.step()
            optimizer.zero_grad()

        total_train_loss += train_loss.item()

    # モデルの評価
    with torch.no_grad():
        for x, _, _ in val_loader:
            x = x.unsqueeze(1)
            model.eval()

            with torch.no_grad():
                x = x.to(device)

                rec_img, mask = model(x)
                valid_loss = torch.mean((rec_img - x) ** 2 * mask) / config["mask_ratio"]

                total_valid_loss += valid_loss.item()


    print(f"Epoch[{epoch+1} / {epochs}] Train Loss: {total_train_loss/len(train_loader):.4f} Valid Loss: {total_valid_loss/len(val_loader):.4f}")


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [04:08<00:00,  1.03it/s]


Epoch[1 / 100] Train Loss: 0.2590 Valid Loss: 0.1784


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [04:11<00:00,  1.02it/s]


Epoch[2 / 100] Train Loss: 0.1938 Valid Loss: 0.1712


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [04:09<00:00,  1.03it/s]


...

Epoch[98 / 100] Train Loss: 0.1243 Valid Loss: 0.1292


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [04:38<00:00,  1.08s/it]


Epoch[99 / 100] Train Loss: 0.1244 Valid Loss: 0.1293


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [04:37<00:00,  1.08s/it]


Epoch[100 / 100] Train Loss: 0.1245 Valid Loss: 0.1292


In [27]:
# モデルを保存しておく
torch.save(model.state_dict(), "model/MAE_pretrain_params_240716_1.pth")
del model, train_loss, valid_loss, train_set, val_set, test_set
torch.cuda.empty_cache()

In [ ]:
# ViT Classifier (adapted from Lecture 9) (240716)

In [55]:
class ViT_Classifier(nn.Module):
    def __init__(self, encoder: MAE_Encoder, num_classes=1854):
        super().__init__()
        self.cls_token = encoder.cls_token
        self.pos_embedding = encoder.pos_embedding
        self.patchify = encoder.patchify
        self.transformer = encoder.transformer
        self.layer_norm = encoder.layer_norm

        """
        self.head = nn.Sequential(
            nn.Linear(self.pos_embedding.shape[-1], num_classes),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(num_classes, num_classes),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(num_classes, num_classes)
        )
        """
        
        self.head = nn.Sequential(
            nn.Linear(self.pos_embedding.shape[-1], num_classes),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(num_classes, num_classes)
        )
        

    def forward(self, img):
        patches = self.patchify(img)
        patches = patches + self.pos_embedding  # positional embedding

        patches = torch.cat([self.cls_token.repeat(patches.shape[0], 1, 1), patches], dim=1)  # class tokenを結合
        features = self.layer_norm(self.transformer(patches))
        logits = self.head(features[:, 0])  # cls tokenのみを入力する
        return logits

    def get_last_selfattention(self, x):
        patches = self.patchify(x)
        patches = patches + self.pos_embedding

        patches = torch.cat([self.cls_token.repeat(patches.shape[0], 1, 1), patches], dim=1)  # class tokenを結合
        for i, block in enumerate(self.transformer):
            if i < len(self.transformer) - 1:
                patches = block(patches)
            else:
                return block(patches, return_attn=True)


class ViT_Classifier_ensemble(nn.Module):
    def __init__(self, encoder: MAE_Encoder, num_classes=1854):
        super().__init__()
        self.cls_token = encoder.cls_token
        self.pos_embedding = encoder.pos_embedding
        self.patchify = encoder.patchify
        self.transformer = encoder.transformer
        self.layer_norm = encoder.layer_norm

        self.head1 = nn.Sequential(
            nn.Linear(self.pos_embedding.shape[-1], num_classes),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(num_classes, num_classes)
        )

        self.head2 = nn.Sequential(
            nn.Linear(self.pos_embedding.shape[-1], num_classes),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(num_classes, num_classes)
        )

        self.head3 = nn.Sequential(
            nn.Linear(self.pos_embedding.shape[-1], num_classes),
            nn.ReLU(),
            nn.Dropout(0.28),
            nn.Linear(num_classes, num_classes)
        )

    def forward(self, img):
        patches = self.patchify(img)
        patches = patches + self.pos_embedding  # positional embedding

        patches = torch.cat([self.cls_token.repeat(patches.shape[0], 1, 1), patches], dim=1)  # class tokenを結合
        features = self.layer_norm(self.transformer(patches))
        logits1 = self.head1(features[:, 0])  # cls tokenのみを入力する
        logits2 = self.head2(features[:, 0])
        logits3 = self.head3(features[:, 0])
        logits = (logits1+logits2+logits3)/3
        return logits

    def get_last_selfattention(self, x):
        patches = self.patchify(x)
        patches = patches + self.pos_embedding

        patches = torch.cat([self.cls_token.repeat(patches.shape[0], 1, 1), patches], dim=1)  # class tokenを結合
        for i, block in enumerate(self.transformer):
            if i < len(self.transformer) - 1:
                patches = block(patches)
            else:
                return block(patches, return_attn=True)

In [56]:
# ハイパーパラメータの設定

subject_index = None

config = {
    "image_size": [271, 280],
    "patch_size": [271, 20],
    "emb_dim": 1854,
    "enc_layers": 24,
    "enc_heads": 8,
    "enc_dim_head": 16,
    "enc_mlp_dim": 1854,
    "dec_layers": 4,
    "dec_heads": 5,
    "dec_dim_head": 16,
    "dec_mlp_dim": 1854,
    "mask_ratio": 0.,
    "dropout": 0.25
}


device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model = MAE_ViT(**config).to(device)
#pretrained_model.load_state_dict(torch.load("model/MAE_pretrain_params_subject"+str(subject_index)+"_240713.pth", map_location=device))
#pretrained_model.load_state_dict(torch.load("model/MAE_pretrain_params_withCLIP_subject"+str(subject_index)+"_240715.pth", map_location=device))
pretrained_model.load_state_dict(torch.load("model/MAE_pretrain_params_240716_1.pth", map_location=device))


encoder = pretrained_model.encoder

# モデルの定義
model = ViT_Classifier(encoder).to(device)


In [10]:

subject_index = None
args = {"batch_size": 256, "epochs": 80, "lr": 0.001, "device": "cuda", "num_workers": 0, "seed": 1234, "use_wandb": False, "data_dir": "data/", "model_path": "A"}

# ------------------
#    Dataloader
# ------------------
loader_args = {"batch_size": args["batch_size"], "num_workers": args["num_workers"]}

train_set = ThingsMEGDataset_selective("train", subject_index, args["data_dir"])
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
val_set = ThingsMEGDataset_selective("val", subject_index, args["data_dir"])
val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)
test_set = ThingsMEGDataset_selective("test", subject_index, args["data_dir"])
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, **loader_args)

100%|███████████████████████████████████████████████████████████████████████████| 16432/16432 [00:17<00:00, 951.71it/s]


In [57]:
device = "cuda" if torch.cuda.is_available() else "cpu"


epochs_csf = 400
lr_csf = 0.001
warmup_length_csf = 25
#optimizer_csf = optim.AdamW(list(model.head1.parameters())+list(model.head2.parameters())+list(model.head3.parameters()), lr=lr_csf, betas=(0.9, 0.999), weight_decay=0.1)  # 分類器部分のみ学習
optimizer_csf = optim.AdamW(model.head.parameters(), lr=lr_csf, betas=(0.9, 0.999), weight_decay=0.05)  # 分類器部分のみ学習
scheduler = CosineScheduler(epochs_csf, lr_csf, warmup_length_csf)
criterion = nn.CrossEntropyLoss()

In [58]:

accuracy = Accuracy(
    task="multiclass", num_classes=train_set.num_classes, top_k=10
).to(device)

best_acc = -1
cnt = 1
# モデルの訓練
for epoch_csf in range(epochs_csf):
    
    total_train_loss_csf = 0.
    total_valid_loss_csf = 0.
    train_loss, train_acc, val_loss, val_acc = [], [], [], []
    step_count = 0

    model.train()
    for x, y, subject_idxs in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        model.zero_grad()
        
        #with torch.no_grad():
        y_pred = model(x.unsqueeze(1))

        train_loss_csf = criterion(y_pred, y)
        total_train_loss_csf += train_loss_csf.item()
        train_loss.append(train_loss_csf.item())
        
        optimizer_csf.zero_grad()
        train_loss_csf.backward()
        
        optimizer_csf.step()

        acc = accuracy(y_pred, y)
        train_acc.append(acc.item())
        
        step_count += 1
    
    model.eval()
    # モデルの評価
    with torch.no_grad():
        for x, y, subject_idxs in val_loader:
            x, y = x.to(device), y.to(device)
    
            y_pred = model(x.unsqueeze(1))

            valid_loss_csf = criterion(y_pred, y)
            val_loss.append(valid_loss_csf)
            val_acc.append(accuracy(y_pred, y).item())

            total_valid_loss_csf += valid_loss_csf.item()

        
    print(f"Epoch[{epoch_csf+1} / {epochs_csf}] Train Loss: {total_train_loss_csf/len(train_loader):.4f} Valid Loss: {total_valid_loss_csf/len(val_loader):.4f} Train Acc: {np.mean(train_acc):.4f}  Valid Acc: {np.mean(val_acc):.4f} ")

    if (np.mean(val_acc)>=best_acc)*(np.mean(val_acc)>=0.015):
        torch.save(model.state_dict(), "final_model_save/model"+str(cnt)+"_acc"+str(np.mean(val_acc))+".pth")
        best_acc = np.mean(val_acc)
        cnt += 1

100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [01:08<00:00,  3.75it/s]


Epoch[1 / 400] Train Loss: 7.5211 Valid Loss: 7.5002 Train Acc: 0.0073  Valid Acc: 0.0096 


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [01:07<00:00,  3.78it/s]


Epoch[2 / 400] Train Loss: 7.5066 Valid Loss: 7.4974 Train Acc: 0.0080  Valid Acc: 0.0096 


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [01:08<00:00,  3.77it/s]


...

Epoch[132 / 400] Train Loss: 7.4956 Valid Loss: 7.4925 Train Acc: 0.0074  Valid Acc: 0.0096 


100%|████████████████████████████████████████████████████████████████████████████████| 257/257 [01:07<00:00,  3.78it/s]


Epoch[133 / 400] Train Loss: 7.4956 Valid Loss: 7.4926 Train Acc: 0.0070  Valid Acc: 0.0099 


 70%|███████████████████████████████████████████████████████▋                        | 179/257 [00:48<00:21,  3.66it/s]


KeyboardInterrupt: 

In [47]:
logdir = ""
preds = [] 
model.eval()
for X, subject_idxs in tqdm(test_loader, desc="Validation"):        
    preds.append(model(X.to(device).unsqueeze(1)).detach().cpu())
    
preds = torch.cat(preds, dim=0).numpy()
np.save(os.path.join(logdir, "submission"), preds)
cprint(f"Submission {preds.shape} saved at {logdir}", "cyan")


Validation: 100%|██████████████████████████████████████████████████████████████████████| 65/65 [00:09<00:00,  7.10it/s]


Submission (16432, 1854) saved at 


In [ ]:
# Model implementation ends here. Further attempts for improvements are shown below this line

In [ ]:
# Training MAEs for individual subjects MEG images WITH CLIP (240713)

In [8]:
for ii in range(4):
    # ハイパーパラメータの設定
    config = {
        "image_size": [271, 280],
        "patch_size": [271, 4],
        "emb_dim": 256,
        "enc_layers": 24,
        "enc_heads": 3,
        "enc_dim_head": 16,
        "enc_mlp_dim": 256,
        "dec_layers": 4,
        "dec_heads": 3,
        "dec_dim_head": 16,
        "dec_mlp_dim": 256,
        "mask_ratio": 0.1,
        "dropout": 0.25
    }
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = MAE_ViT(**config).to(device)
    model.load_state_dict(torch.load("model/MAE_pretrain_params_240713.pth", map_location=device))


    # load CLIP
    device = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model = "ViT-L/14"
    CLIP, preprocess = clip.load(clip_model, device=device)

    PL_model = BasicConvProjector(768, 255, 64).to(device)


    args = {"batch_size": 128, "device": "cuda", "num_workers": 0, "seed": 1234, "use_wandb": False, "data_dir": "data/"}
    img_path = "images-002/images/"
    # ------------------
    #    Dataloader
    # ------------------
    
    sj_idx = ii
    img_save_path = "save_imgs/"
    loader_args = {"batch_size": args["batch_size"], "num_workers": args["num_workers"]}
    train_set = ThingsMEGDataset5_PreloadSplit(preprocess, "train", sj_idx, True, args["data_dir"], img_path, clip_model, img_save_path, device, False)
    print("train_set loaded")
    train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
    val_set = ThingsMEGDataset5_PreloadSplit(preprocess, "val", sj_idx, True, args["data_dir"], img_path, clip_model, img_save_path, device, False)
    print("val_set loaded")
    val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)
    #test_set = ThingsMEGDataset5_PreloadSplit(preprocess, "test", sj_idx, True, args["data_dir"], img_path, clip_model, img_save_path, device, False)
    #print("test_set loaded")
    #test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, batch_size=args["batch_size"], num_workers=args["num_workers"])
        

    
    
    epochs = 100
    lr = 0.0001
    step_count = 0
    warmup_length = 25
    lbd = 0.08
    optimizer = optim.AdamW(list(model.parameters())+list(PL_model.parameters()), lr=lr, betas=(0.9, 0.95), weight_decay=5e-5)
    scheduler = CosineScheduler(epochs, lr, warmup_length)
    cossim = nn.CosineSimilarity(dim=1, eps=1e-6)
    scheduler = CosineScheduler(epochs, lr, warmup_length)


    for epoch in range(epochs):
        # スケジューラで学習率を更新する
        new_lr = scheduler(epoch)
        set_lr(new_lr, optimizer)
    
        total_train_loss_MAE = 0.
        total_valid_loss_MAE = 0.

        total_train_loss_CLIP = 0.
        total_valid_loss_CLIP = 0.

        total_train_loss = 0.
        total_valid_loss = 0.
    
        # モデルの訓練
        model.train()
        CLIP.eval()
        PL_model.train()
        for x, _, _, image_features  in tqdm(train_loader):
            x = x.unsqueeze(1)
            step_count += 1
            x = x.to(device)
            image_features = image_features.to(device)
    
            rec_img, mask = model(x)
            train_loss_MAE = torch.mean((rec_img - x) ** 2 * mask) / config["mask_ratio"]

            #with torch.no_grad():
            features, bk_idx = model.encoder(x)
            meg_features = PL_model(features)
            
            train_loss_CLIP = 1-torch.mean(cossim(meg_features, image_features))
            
            
            train_loss = train_loss_MAE + lbd*train_loss_CLIP
            train_loss.backward()
        
            if step_count % 8 == 0:  # 8イテレーションごとに更新することで，擬似的にバッチサイズを大きくしている
                optimizer.step()
                optimizer.zero_grad()

            total_train_loss_MAE += train_loss_MAE.item()
            total_train_loss_CLIP += train_loss_CLIP.item()
            total_train_loss += train_loss.item()
    
        # モデルの評価
        with torch.no_grad():
            model.eval()
            PL_model.eval()
            for x, _, _, image_features in val_loader:
                x = x.to(device)
                x = x.unsqueeze(1)
                image_features = image_features.to(device)

                rec_img, mask = model(x)
                valid_loss_MAE = torch.mean((rec_img - x) ** 2 * mask) / config["mask_ratio"]

                features, bk_idx = model.encoder(x)
                meg_features = PL_model(features)

                valid_loss_CLIP = 1-torch.mean(cossim(meg_features, image_features))

                valid_loss = train_loss_MAE + lbd*train_loss_CLIP

                total_valid_loss_MAE += valid_loss_MAE.item()
                total_valid_loss_CLIP += valid_loss_CLIP.item()
                total_valid_loss += valid_loss.item()

        
    
        print(f"Epoch[{epoch+1} / {epochs}] Train Loss (MAE): {total_train_loss_MAE/len(train_loader):.4f} Valid Loss (MAE): {total_valid_loss_MAE/len(val_loader):.4f} Train Loss (CLIP): {total_train_loss_CLIP/len(train_loader):.4f} Valid Loss (CLIP): {total_valid_loss_CLIP/len(val_loader):.4f} Train Loss (Total): {total_train_loss/len(train_loader):.4f} Valid Loss (Total): {total_valid_loss/len(val_loader):.4f}")






    torch.save(model.state_dict(), "model/MAE_pretrain_params_withCLIP_subject_240717.pth")
    del model, CLIP, PL_model, train_loss, valid_loss, train_set, val_set
    torch.cuda.empty_cache()







Filtering for train: 100%|██████████████████████████████████████████████████████| 16432/16432 [00:17<00:00, 956.28it/s]


train_set loaded


Filtering for val: 100%|██████████████████████████████████████████████████████████| 4108/4108 [00:04<00:00, 956.89it/s]


val_set loaded


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:40<00:00,  3.20it/s]


...

Epoch[98 / 100] Train Loss (MAE): 0.0426 Valid Loss (MAE): 0.0348 Train Loss (CLIP): 0.6592 Valid Loss (CLIP): 0.4660 Train Loss (Total): 0.0953 Valid Loss (Total): 0.0985


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:39<00:00,  3.25it/s]


Epoch[99 / 100] Train Loss (MAE): 0.0426 Valid Loss (MAE): 0.0351 Train Loss (CLIP): 0.6593 Valid Loss (CLIP): 0.4660 Train Loss (Total): 0.0953 Valid Loss (Total): 0.0953


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [00:39<00:00,  3.27it/s]


Epoch[100 / 100] Train Loss (MAE): 0.0426 Valid Loss (MAE): 0.0349 Train Loss (CLIP): 0.6596 Valid Loss (CLIP): 0.4660 Train Loss (Total): 0.0954 Valid Loss (Total): 0.0966


In [ ]:
# Training MAEs for individual subjects MEG images (240713)

In [14]:
for ii in range(4):

    args = {"batch_size": 256, "epochs": 80, "lr": 0.001, "device": "cuda", "num_workers": 0, "seed": 1234, "use_wandb": False, "data_dir": "data/", "model_path": "A"}
    
    # ------------------
    #    Dataloader
    # ------------------
    loader_args = {"batch_size": args["batch_size"], "num_workers": args["num_workers"]}
    
    train_set = ThingsMEGDataset_selective("train", ii, args["data_dir"])
    train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
    val_set = ThingsMEGDataset_selective("val", ii, args["data_dir"])
    val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)

    
    # ハイパーパラメータの設定
    config = {
        "image_size": [271, 280],
        "patch_size": [271, 4],
        "emb_dim": 256,
        "enc_layers": 24,
        "enc_heads": 3,
        "enc_dim_head": 16,
        "enc_mlp_dim": 256,
        "dec_layers": 4,
        "dec_heads": 3,
        "dec_dim_head": 16,
        "dec_mlp_dim": 256,
        "mask_ratio": 0.1,
        "dropout": 0.25
    }
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = MAE_ViT(**config).to(device)
    model.load_state_dict(torch.load("model/MAE_pretrain_params_with_240713.pth", map_location=device))

    epochs = 500
    lr = 0.00005
    step_count = 0
    warmup_length = 80
    batch_size = 128
    optimizer = optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.95), weight_decay=0.05)
    scheduler = CosineScheduler(epochs, lr, warmup_length)


    for epoch in range(epochs):
        # スケジューラで学習率を更新する
        new_lr = scheduler(epoch)
        set_lr(new_lr, optimizer)
    
        total_train_loss = 0.
        total_valid_loss = 0.
    
        # モデルの訓練
        for x, _, _ in tqdm(train_loader):
            x = x.unsqueeze(1)
            step_count += 1
            model.train()
            x = x.to(device)
    
            rec_img, mask = model(x)
            train_loss = torch.mean((rec_img - x) ** 2 * mask) / config["mask_ratio"]
            train_loss.backward()
    
            if step_count % 8 == 0:  # 8イテレーションごとに更新することで，擬似的にバッチサイズを大きくしている
                optimizer.step()
                optimizer.zero_grad()
    
            total_train_loss += train_loss.item()
    
        # モデルの評価
        with torch.no_grad():
            for x, _, _ in val_loader:
                x = x.unsqueeze(1)
                model.eval()
    
                with torch.no_grad():
                    x = x.to(device)
    
                    rec_img, mask = model(x)
                    valid_loss = torch.mean((rec_img - x) ** 2 * mask) / config["mask_ratio"]
    
                    total_valid_loss += valid_loss.item()
    
    
        print(f"Epoch[{epoch+1} / {epochs}] Train Loss: {total_train_loss/len(train_loader):.4f} Valid Loss: {total_valid_loss/len(val_loader):.4f}")






    torch.save(model.state_dict(), "model/MAE_pretrain_params_subject"+str(ii)+"_240713.pth")
    del model, train_loss, valid_loss, train_set, val_set
    torch.cuda.empty_cache()

100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:07<00:00,  8.63it/s]


Epoch[1 / 500] Train Loss: 0.0411 Valid Loss: 0.0338


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:07<00:00,  8.91it/s]


Epoch[2 / 500] Train Loss: 0.0411 Valid Loss: 0.0336


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:07<00:00,  8.93it/s]


...

Epoch[498 / 500] Train Loss: 0.0420 Valid Loss: 0.0341


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:07<00:00,  8.86it/s]


Epoch[499 / 500] Train Loss: 0.0422 Valid Loss: 0.0344


100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:07<00:00,  8.89it/s]


Epoch[500 / 500] Train Loss: 0.0423 Valid Loss: 0.0346


In [ ]:
# CLIP with MAE training enabled (240715)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = "ViT-L/14"
CLIP, preprocess = clip.load(clip_model, device=device)
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [8]:
args = {"batch_size": 128, "device": "cuda", "num_workers": 0, "seed": 1234, "use_wandb": False, "data_dir": "data/"}
img_path = "images-002/images/"
# ------------------
#    Dataloader
# ------------------

sj_idx = 0
img_save_path = "save_imgs/"
loader_args = {"batch_size": args["batch_size"], "num_workers": args["num_workers"]}
train_set = ThingsMEGDataset5_PreloadSplit(preprocess, "train", sj_idx, True, args["data_dir"], img_path, clip_model, img_save_path, device, False)
print("train_set loaded")
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
val_set = ThingsMEGDataset5_PreloadSplit(preprocess, "val", sj_idx, True, args["data_dir"], img_path, clip_model, img_save_path, device, False)
print("val_set loaded")
val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)
test_set = ThingsMEGDataset5_PreloadSplit(preprocess, "test", sj_idx, True, args["data_dir"], img_path, clip_model, img_save_path, device, False)
print("test_set loaded")
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, batch_size=args["batch_size"], num_workers=args["num_workers"])

Filtering for train: 100%|██████████████████████████████████████████████████████| 16432/16432 [00:17<00:00, 934.46it/s]
Compressing images for train: 16432it [10:52, 25.20it/s]


train_set loaded


Filtering for val: 100%|██████████████████████████████████████████████████████████| 4108/4108 [00:04<00:00, 894.18it/s]
Compressing images for val: 4108it [02:43, 25.11it/s]


val_set loaded


Filtering for test:  13%|███████▌                                                | 2211/16432 [00:02<00:15, 914.94it/s]


ValueError: could not convert b, a, and x to a common type

In [33]:
# ハイパーパラメータの設定
config = {
    "image_size": [271, 280],
    "patch_size": [271, 4],
    "emb_dim": 256,
    "enc_layers": 24,
    "enc_heads": 3,
    "enc_dim_head": 16,
    "enc_mlp_dim": 256,
    "dec_layers": 4,
    "dec_heads": 3,
    "dec_dim_head": 16,
    "dec_mlp_dim": 256,
    "mask_ratio": 0.1,
    "dropout": 0.25
}

model = MAE_ViT(**config).to(device)
model.load_state_dict(torch.load("model/MAE_pretrain_params_subject0_240713.pth", map_location=device))
PL_model = BasicConvProjector(768, 255, 64).to(device)

In [34]:
epochs_CLIP = 100
lr_CLIP = 0.0024
step_count_CLIP = 0
warmup_length_CLIP = 25
optimizer_CLIP = optim.AdamW(list(model.parameters())+list(PL_model.parameters()), lr=lr_CLIP, betas=(0.9, 0.95), weight_decay=5e-5)
scheduler_CLIP = CosineScheduler(epochs_CLIP, lr_CLIP, warmup_length_CLIP)
cossim = nn.CosineSimilarity(dim=1, eps=1e-6)

In [35]:
# モデルの訓練
img_dir = "images-002/images/"

lr_list = []
for epoch_CLIP in range(epochs_CLIP):
    # スケジューラで学習率を更新する
    new_lr_CLIP = scheduler_CLIP(epochs_CLIP)
    lr_list.append(new_lr_CLIP)
    set_lr(new_lr_CLIP, optimizer_CLIP)
    
    total_train_loss_CLIP = 0.
    total_valid_loss_CLIP = 0.

    step_count = 0

    model.train()
    CLIP.eval()
    PL_model.train()
    for x, _, _, image_features in tqdm(train_loader):
        x = x.to(device)
        model.zero_grad()
        PL_model.zero_grad()
        
        #with torch.no_grad():
        features, bk_idx = model.encoder(x.unsqueeze(1))

        #features = features.unsqueeze(1)
        meg_features = PL_model.forward(features)
        
        # Image Encoder (CLIP)
        #images = images.to(device)
        #image_features = CLIP.encode_image(images)
        image_features = image_features.to(device)

        train_loss_CLIP = 1-torch.mean(cossim(meg_features, image_features))
        #with torch.no_grad():
        train_loss_CLIP.backward()
    
        total_train_loss_CLIP += train_loss_CLIP.item()
    
        if step_count % 8 == 0:  # 8イテレーションごとに更新することで，擬似的にバッチサイズを大きくしている
            optimizer_CLIP.step()
            optimizer_CLIP.zero_grad()
        
        step_count += 1
    
    model.eval()
    PL_model.eval()
    # モデルの評価
    with torch.no_grad():
        for x, _, _, image_features in val_loader:
            with torch.no_grad():
                x = x.to(device)
    
                features, bk_idx = model.encoder(x.unsqueeze(1))
                meg_features = PL_model(features)

                # Image Encoder (CLIP)
                #images = images.to(device)
                #image_features = CLIP.encode_image(images)
                image_features = image_features.to(device)
    
                valid_loss_CLIP = 1-torch.mean(cossim(meg_features, image_features))
    
                total_valid_loss_CLIP += valid_loss_CLIP.item()
    
        
    print(f"Epoch[{epoch_CLIP+1} / {epochs_CLIP}] Train Loss: {total_train_loss_CLIP/len(train_loader):.4f} Valid Loss: {total_valid_loss_CLIP/len(val_loader):.4f}")




100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [01:08<00:00,  1.88it/s]


Epoch[1 / 100] Train Loss: 0.7108 Valid Loss: 0.4842


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [01:08<00:00,  1.89it/s]


Epoch[2 / 100] Train Loss: 0.6791 Valid Loss: 0.4780


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [01:08<00:00,  1.90it/s]


...

Epoch[98 / 100] Train Loss: 0.6602 Valid Loss: 0.4675


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [01:08<00:00,  1.89it/s]


Epoch[99 / 100] Train Loss: 0.6594 Valid Loss: 0.4674


100%|████████████████████████████████████████████████████████████████████████████████| 129/129 [01:08<00:00,  1.89it/s]


Epoch[100 / 100] Train Loss: 0.6614 Valid Loss: 0.4674


In [36]:
# モデルを保存しておく
torch.save(model.state_dict(), "model/MAE_pretrain_params_subject0_CLIP_body_240715.pth")
torch.save(PL_model.state_dict(), "model/MAE_pretrain_params_subject0_CLIP_head_240715.pth")
del model, PL_model, train_loss, valid_loss, train_set, val_set, test_set

NameError: name 'train_loss' is not defined

In [21]:
torch.save(PL_model.state_dict(), "model/ProjectionLayer_params.pth")